<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/mmlu_pro/category_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install langchain-upstage langchain-community pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: langchain-core


In [3]:
pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=223279f86901cef1a4be7223edb238bdb62b75b201a61521c862e21b00635b74
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


In [4]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 35.7 MB/s eta 0:00:00


In [5]:
import os
import json
import re
from typing import List, Tuple

from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

import wikipediaapi
import pandas as pd

In [7]:
cd /content/drive/MyDrive/Colab Notebooks/2025_2/nlp

/content/drive/MyDrive/Colab Notebooks/2025_2/nlp


In [9]:
df = pd.read_csv("./dataset/testset.csv")

In [10]:
UPSTAGE_API_KEY = "up_g7T2cQoLKZH6Oi2n4MHOW706XAdSs"

In [11]:
llm_classifier = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1, #거의 항상 같은 결과, 안정적이게
)

In [12]:
llm_solver = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.2, #좀 더 유연한 답변을 내도록
)

In [13]:
category_index = {
    "law":        "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/mmlu_category/law",
    "psychology": "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/mmlu_category/psychology",
    "business":   "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/mmlu_category/business",
    "philosophy": "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/mmlu_category/philosophy",
    "history":    "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/mmlu_category/history",
}
categories = list(category_index.keys())

In [14]:
emb = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large")

# 카테고리별 faiss 한번에 로드해서 캐시
vectorstore = {}
for cat, path in category_index.items():
    vectorstore[cat] = FAISS.load_local(
        folder_path=path,
        embeddings=emb,
        allow_dangerous_deserialization=True,
    )

wiki = wikipediaapi.Wikipedia(user_agent= "NLP-RAG/1.0", language = 'en')

**1st LLM**:

- return category

- return 3 keywords

In [15]:
category_prompt_template = """
You are an expert exam classifier for MMLU-Pro

There are 5 possible categories:
- law
- psychology
- business
- philosophy
- history

[Task]
Given a multiple-choice exam question (including all options),
1. Choose one best category from the list above.
2. Extract exactly 3 important keywords (one noun that consists of single word or short phrase)
   that will be useful to search textbooks and Wikipedia.
   Keywords should be as specific and accurate as possible (e.g., "consent",
   "cognitive dissonance", "Keynesian economics").

[Output format]
Return a valid JSON with the following fields:
- "category": one of ["law","psychology","business","philosophy","history"]
- "keywords": a list of exactly 3 strings

This is an output example:
{{
  "category": "psychology",
  "keywords": ["informed consent", "assent", "child counseling"]
}}

[Question]
{question}
""".strip()

category_prompt = ChatPromptTemplate.from_template(category_prompt_template)

In [30]:
def classify_and_extract_keywords(question_text: str):
    messages = category_prompt.format_messages(question=question_text)
    resp = llm_classifier.invoke(messages)
    raw = resp.content.strip()

    if not raw:
        # 완전 빈 응답이면 기본값으로 대충 처리 (죽지 않게)
        print("[WARN] Empty LLM output for category, fallback to 'history'")
        return "history", ["history", "war", "europe"]

    # 1차 시도: 전체를 JSON으로 해석
    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        # 2차 시도: 문자열 안에서 {...} 구간만 잘라서 해석
        start = raw.find("{")
        end = raw.rfind("}")
        if start == -1 or end == -1:
            print("[WARN] No JSON object found in output, fallback to 'history'")
            return "history", ["history", "war", "europe"]

        json_str = raw[start:end+1]
        try:
            data = json.loads(json_str)
        except json.JSONDecodeError as e:
            print("[WARN] JSON parse failed again:", e)
            return "history", ["history", "war", "europe"]

    category = data.get("category", "history").strip().lower()
    keywords = data.get("keywords", [])
    keywords = [str(k).strip() for k in keywords][:3]

    if not keywords:
        keywords = ["history", "war", "europe"]

    return category, keywords

In [31]:
def fetch_wiki_context(keywords, max_chars_per_page=1200):
    snippets = []
    for kw in keywords:
        try:
            page = wiki.page(kw)
            if not page.exists():
                print(f"[WARN] page for '{kw}' does not exist")
                continue
            text = page.text[:max_chars_per_page]
            snippets.append(f"[Wikipedia: {page.title}]\n{text}")
        except Exception as e:
            print(f"[WARN] wikipediaapi fetch failed for '{kw}': {e}")
            continue

    return "\n\n".join(snippets)

In [32]:
TOP_K_TEXTBOOK = 5

def build_context_for_solver(question_text: str, category: str, keywords):
    # 1) 카테고리별 textbook RAG
    vs = vectorstore[category]
    docs = vs.similarity_search(question_text, k=TOP_K_TEXTBOOK)
    textbook_context = "\n\n".join(
        f"[Textbook Doc {i+1}] {d.page_content}" for i, d in enumerate(docs)
    )

    # 2) Wikipedia context
    wiki_context = fetch_wiki_context(keywords)

    full_context = f"""\
=== TEXTBOOK CONTEXT ({category}) ===
{textbook_context}

=== WIKIPEDIA CONTEXT (keywords: {', '.join(keywords)}) ===
{wiki_context}
"""
    return full_context

**2nd LLM**:

- return CoT
- return 최종 정답

# Prompt Template

In [33]:
SOLVER_PROMPT_LAW_TMPL = """
You are an expert law exam solver. You specialize in common-law style multiple-choice questions
MMLU-Pro (criminal law, torts, contracts, property, evidence, constitutional law, etc.).

You are given two inputs:

[CONTEXT]
{context}

[QUESTION]
{question}

Use the CONTEXT as textbook + Wikipedia support, but ALWAYS follow these principles:

1) PRIORITIZE THE QUESTION FACT PATTERN
- First, read the QUESTION (including facts and answer options) very carefully.
- Treat the facts and wording in the QUESTION as the primary authority.
- Do NOT contradict the facts as stated in the QUESTION.
- If the CONTEXT conflicts with the QUESTION, you MUST follow the QUESTION.

2) EXTRACT KEY LEGAL CLUES FROM THE QUESTION
Before judging the options, silently identify 3–5 essential legal clues from the QUESTION itself, such as:
- relevant area of law (e.g., common-law robbery, larceny, attempt, accomplice liability, defenses),
- key facts about timing, knowledge, intent (mens rea), consent, force, or possession,
- jurisdiction assumptions (assume standard U.S. common law / majority rule unless the CONTEXT clearly says otherwise).

Base your reasoning on these clues.

3) USE CONTEXT CAREFULLY (TEXTBOOK > WIKIPEDIA)
- Use the CONTEXT to recall or confirm black-letter rules, elements, and standard doctrines.
- If textbook material and Wikipedia conflict, trust the textbook-style explanation in the CONTEXT.
- Do NOT narrow or expand a rule beyond what is normally accepted in standard common law / majority doctrine,
  unless the CONTEXT explicitly tells you to use a specific variant.

4) APPLY BLACK-LETTER LAW PRECISELY
- Identify the relevant rule(s) (e.g., elements of common-law larceny, robbery, burglary, receiving stolen property,
  attempt, conspiracy, self-defense, etc.).
- Apply every element to the facts in the QUESTION.
- Pay careful attention to:
  - timing of force or threats,
  - whether the defendant knew property was stolen,
  - whether there was a trespassory taking,
  - whether possession was obtained by fraud, trick, or without consent,
  - whether any required mental state (intent, knowledge, recklessness, negligence) is clearly present.

5) COMPARE EACH OPTION STRICTLY
For EACH answer choice:
- Explain briefly (in your own reasoning) why it is correct or incorrect,
  always referencing specific facts from the QUESTION and the applicable legal rule.
- When TWO OR MORE options seem partially correct:
  * Prefer the option that:
    - best matches the precise legal definition and all elements,
    - correctly reflects the timing and mental state required by the rule,
    - does NOT add extra facts not in the QUESTION,
    - fits both the QUESTION facts and the CONTEXT.

6) AVOID HALLUCINATIONS
- Do NOT assume that the defendant “knows” something unless the QUESTION clearly states it or it is a necessary inference.
- Do NOT invent additional conduct (e.g., extra threats, agreements, or prior plans) that are not described.
- Do NOT change the jurisdiction, legal standard, or time period unless the CONTEXT or QUESTION explicitly tells you to.
- If the context is incomplete, choose the MOST reasonable answer but do not contradict the given evidence.

7) STYLE AND FINAL ANSWER FORMAT
- Write your reasoning in plain text sentences.
- Do NOT use Markdown formatting (no bullet points, no headings, no **bold**, no code blocks).
- At the very end, on a NEW LINE, output the final answer in the exact format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).
- Do NOT wrap this line in Markdown or quotes.
- Do NOT add any extra text before or after the 'Final Answer: X' line.
- The 'Final Answer: X' line MUST appear only once at the very end.

Now, following all of the above rules, reason step by step using the QUESTION and CONTEXT, then give the final answer.
""".strip()

solver_prompt_law = ChatPromptTemplate.from_template(SOLVER_PROMPT_LAW_TMPL)

In [34]:
SOLVER_PROMPT_PHILOSOPHY_TMPL = """
You are an expert philosophy exam solver. You specialize in advanced multiple-choice questions
(undergraduate and graduate level) in ethics, meta-ethics, epistemology, metaphysics, logic,
philosophy of mind and language, political philosophy, and the history of philosophy
(ancient, medieval, modern, and contemporary).

You are given two inputs:

[CONTEXT]
{context}

[QUESTION]
{question}

The CONTEXT acts like textbook + Wikipedia support. The QUESTION is the primary source.
Follow these rules very carefully:

1) PRIORITIZE THE QUESTION TEXT
- First, read the QUESTION (including any passage and all answer options) very carefully.
- Treat the text and wording of the QUESTION as the strongest evidence.
- If the CONTEXT conflicts with the QUESTION, you MUST follow the QUESTION.

2) EXTRACT KEY PHILOSOPHICAL CLUES FROM THE QUESTION
When solving the problem, follow this procedure:
1. Look only at the [QUESTION] text first and silently extract 3–5 essential philosophical clues, such as:
   - the main concept (e.g., categorical imperative, utilitarianism, internalism vs externalism,
     Gettier problem, a priori vs a posteriori, necessary vs sufficient conditions),
   - the relevant philosopher or school (e.g., Kant, Mill, Hume, Plato, Aristotle, Rawls, behaviorism, logical positivism),
   - the logical structure (e.g., what follows from what, which statement would they reject, which principle is violated).
2. Then, look at these clues and the CONTEXT and compare them.
   - If the logic between the QUESTION clues and the CONTEXT conflicts, trust the QUESTION clues.
3. If the clues within textbook-style parts of the CONTEXT and Wikipedia-style parts are conflicted,
   trust the textbook-style exposition in the CONTEXT.

3) USE CONTEXT TO RECALL DEFINITIONS, NOT TO OVERRIDE THE QUESTION
- Use the CONTEXT to recall standard definitions, distinctions, and canonical views of philosophers.
- Do NOT invent new doctrines or attribute views to philosophers that are not well-established.
- Apply the standard reading unless the CONTEXT explicitly specifies a different interpretation.

4) APPLY PHILOSOPHICAL DOCTRINES PRECISELY
- Identify which theory, argument, or distinction the question is really testing.
- Pay attention to:
  - necessary vs sufficient conditions,
  - analytic vs synthetic, a priori vs a posteriori,
  - validity vs soundness,
  - deontological vs consequentialist reasoning,
  - internal vs external justification,
  - the exact wording of a principle or formulation (e.g., Kant’s Humanity formulation, Mill’s harm principle).
- For “Which of the following would X most likely endorse/reject?” style questions, base your choice on
  X’s actual doctrine, not on your own judgment of what is true.

5) COMPARE EACH OPTION CAREFULLY
For EACH option:
- Briefly evaluate why it could be correct or incorrect, using explicit references to:
  - the key clues you extracted from the QUESTION, and
  - the relevant definitions or doctrines from the CONTEXT.
- When TWO OR MORE options seem partially correct:
  * Prefer the option that:
    - best matches the canonical definition or the philosopher’s standard view,
    - captures the fundamental idea of the theory rather than a peripheral detail,
    - does not add extra assumptions not stated in the QUESTION,
    - uses the correct logical strength (e.g., does not turn “some” into “all,” does not confuse necessary with sufficient).

6) AVOID HALLUCINATIONS
- Do NOT assume extra premises, background facts, or historical claims that are not stated or strongly implied.
- Do NOT change the content of a doctrine to make an option fit better; respect standard textbook interpretations.
- If the context is incomplete, choose the MOST reasonable answer that fits BOTH the QUESTION and the reliable parts of the CONTEXT,
  and do not contradict the given evidence.

7) STYLE AND FINAL ANSWER FORMAT
- Write your reasoning in plain text sentences (no bullet points, no headings, no Markdown formatting like ** or #).
- Be concise but logically clear.
- At the very end, on a NEW LINE, output the final answer in the exact format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).
- Do NOT wrap this line in Markdown or quotes.
- Do NOT add any extra text before or after the 'Final Answer: X' line.
- The 'Final Answer: X' line MUST appear only once at the very end.

Now, following all of the above rules, reason step by step using the QUESTION and CONTEXT, then give the final answer.
""".strip()

solver_prompt_philosophy = ChatPromptTemplate.from_template(SOLVER_PROMPT_PHILOSOPHY_TMPL)

In [35]:
SOLVER_PROMPT_PSYCHOLOGY_TMPL = """
You are an expert psychology exam solver. You specialize in advanced multiple-choice questions
(undergraduate and graduate level) in clinical psychology, cognitive psychology, social psychology,
developmental psychology, personality, biological/physiological psychology, learning, psychometrics,
research methods, and ethics (e.g., APA code of conduct).

You are given two inputs:

[CONTEXT]
{context}

[QUESTION]
{question}

The CONTEXT acts like textbook + Wikipedia support. The QUESTION is the primary source.
Follow these rules very carefully:

1) PRIORITIZE THE QUESTION TEXT
- First, read the QUESTION (including any vignette, description, and all answer options) very carefully.
- Treat the facts and wording in the QUESTION as the strongest evidence.
- If the CONTEXT conflicts with the QUESTION, you MUST follow the QUESTION.

2) EXTRACT KEY PSYCHOLOGICAL CLUES FROM THE QUESTION
When solving the problem, follow this procedure:
1. Look only at the [QUESTION] text first and silently extract 3–5 essential psychological clues, such as:
   - the main construct or theory (e.g., classical conditioning, operant conditioning, working memory,
     attachment style, Big Five traits, self-efficacy, cognitive dissonance),
   - the relevant theorist or model (e.g., Piaget, Vygotsky, Erikson, Kohlberg, Bandura, Beck),
   - the developmental stage or level (e.g., preconventional vs conventional, sensorimotor vs formal operational),
   - the type of design or method (e.g., experiment vs correlational study, longitudinal vs cross-sectional),
   - the key symptoms, behaviors, or ethical requirements described in the vignette.
2. Then, look at these clues and the CONTEXT and compare them.
   - If the logic between the QUESTION clues and the CONTEXT conflicts, trust the QUESTION clues.
3. If the clues within textbook-style parts of the CONTEXT and Wikipedia-style parts are conflicted,
   trust the textbook-style exposition in the CONTEXT.

3) USE CONTEXT TO RECALL DEFINITIONS, NOT TO OVERRIDE THE QUESTION
- Use the CONTEXT to recall standard definitions, diagnostic criteria (at a conceptual level),
  major theories, stages, and classical experiments.
- Do NOT invent new constructs or attribute theories to psychologists who are not associated with them.
- Apply the standard textbook interpretation unless the CONTEXT explicitly specifies a different one.

4) APPLY PSYCHOLOGICAL THEORIES PRECISELY
- Identify which theory, construct, stage, or principle the question is really testing.
- Pay attention to:
  - necessary features vs incidental details of a concept,
  - differences between similar concepts (e.g., negative reinforcement vs punishment,
    state vs trait, reliability vs validity),
  - differences between stages (e.g., Kohlberg’s Stage 1 vs Stage 2, Erikson’s crises),
  - the direction of cause/effect or prediction implied by the theory.
- For ethics questions (e.g., APA), carefully consider:
  - informed consent, confidentiality, competence, dual relationships, and risk of harm,
  - what the code obligates the psychologist to do in the specific situation.

5) COMPARE EACH OPTION CAREFULLY
For EACH answer choice:
- Briefly evaluate why it could be correct or incorrect, using explicit references to:
  - the key clues you extracted from the QUESTION, and
  - the relevant definitions or theories from the CONTEXT.
- When TWO OR MORE options seem partially correct:
  * Prefer the option that:
    - best matches the canonical textbook definition or theory,
    - directly fits ALL key facts given in the QUESTION (age, setting, behavior, stated intentions),
    - does not rely on extra assumptions not stated in the QUESTION,
    - has the correct level of generality (not too narrow, not too broad) for the construct or stage.

6) AVOID HALLUCINATIONS
- Do NOT assume additional symptoms, diagnoses, or background history that are not clearly stated.
- Do NOT assume the person has specific knowledge or intentions beyond what the QUESTION describes.
- Do NOT guess rare or exotic explanations when a standard, well-known theory clearly fits better.
- If the context is incomplete, choose the MOST reasonable answer that fits BOTH the QUESTION and the reliable parts of the CONTEXT,
  and do not contradict the given evidence.

7) STYLE AND FINAL ANSWER FORMAT
- Write your reasoning in plain text sentences (no bullet points, no headings, no Markdown formatting like ** or #).
- Be concise but logically clear.
- At the very end, on a NEW LINE, output the final answer in the exact format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).
- Do NOT wrap this line in Markdown or quotes.
- Do NOT add any extra text before or after the 'Final Answer: X' line.
- The 'Final Answer: X' line MUST appear only once at the very end.

Now, following all of the above rules, reason step by step using the QUESTION and CONTEXT, then give the final answer.
""".strip()

solver_prompt_psychology = ChatPromptTemplate.from_template(SOLVER_PROMPT_PSYCHOLOGY_TMPL)

In [36]:
SOLVER_PROMPT_BUSINESS_TMPL = """
You are an expert business and economics exam solver. You specialize in advanced multiple-choice questions
(undergraduate and MBA level) in microeconomics, macroeconomics, finance, accounting, management,
organizational behavior, marketing, operations, business ethics, and quantitative methods.

You are given two inputs:

[CONTEXT]
{context}

[QUESTION]
{question}

The CONTEXT acts like textbook + Wikipedia support. The QUESTION is the primary source.
Follow these rules very carefully:

1) PRIORITIZE THE QUESTION TEXT
- First, read the QUESTION (including any tables, numbers, and all answer options) very carefully.
- Treat the facts, numerical data, and wording in the QUESTION as the strongest evidence.
- If the CONTEXT conflicts with the QUESTION, you MUST follow the QUESTION.

2) EXTRACT KEY BUSINESS / ECON CLUES FROM THE QUESTION
When solving the problem, follow this procedure:
1. Look only at the [QUESTION] text first and silently extract 3–5 essential clues, such as:
   - the main domain (e.g., microeconomics, macroeconomics, corporate finance, accounting, management, marketing),
   - key concepts (e.g., opportunity cost, elasticity, present value, NPV, CAPM, break-even point,
     comparative advantage, marginal cost/benefit, principal–agent problem, game theory, externalities),
   - relevant formulas or relationships (e.g., PV = CF/(1+r)^t, profit = TR – TC, elasticity definitions,
     accounting identities, basic statistics),
   - organizational/management concepts (e.g., leadership style, motivation theory, decision-making biases),
   - business ethics or corporate governance principles.
2. Then, look at these clues and the CONTEXT and compare them.
   - If the logic between the QUESTION clues and the CONTEXT conflicts, trust the QUESTION clues.
3. If the clues within textbook-style parts of the CONTEXT and Wikipedia-style parts are conflicted,
   trust the textbook-style exposition in the CONTEXT.

3) USE CONTEXT TO RECALL DEFINITIONS AND FORMULAS, NOT TO OVERRIDE THE QUESTION
- Use the CONTEXT to recall standard definitions, formulas, and conceptual relationships.
- Do NOT invent new theories or assume non-standard definitions.
- Apply the standard textbook interpretation unless the CONTEXT explicitly specifies a different one.

4) APPLY BUSINESS / ECONOMIC LOGIC PRECISELY
- Identify which concept, model, or calculation the question is really testing.
- Pay careful attention to:
  - the difference between average vs marginal, stock vs flow, nominal vs real, short run vs long run,
  - risk vs return, cost of capital, discounting vs compounding,
  - accounting distinctions (asset/liability/equity, revenue vs profit, cash vs accrual),
  - supply–demand shifts vs movement along a curve,
  - game-theoretic reasoning (dominant strategies, Nash equilibrium),
  - basic statistical reasoning (mean, variance, correlation vs causation).
- When numbers are given, use them logically and consistently.

5) COMPARE EACH OPTION CAREFULLY
For EACH answer choice:
- Briefly evaluate why it could be correct or incorrect, using explicit references to:
  - the key clues you extracted from the QUESTION, and
  - the relevant definitions, models, or formulas from the CONTEXT.
- When TWO OR MORE options seem partially correct:
  * Prefer the option that:
    - best matches the exact definitions and standard formulas,
    - correctly uses all given information in the QUESTION (including signs, units, and constraints),
    - does not rely on extra assumptions not stated in the QUESTION,
    - has the correct economic or managerial interpretation (no confusion of cause and effect, or average vs marginal).

6) AVOID HALLUCINATIONS
- Do NOT assume additional data (e.g., extra numbers, extra constraints) that are not given.
- Do NOT assume a different market structure, time period, or financial environment unless clearly stated.
- Do NOT change definitions (e.g., of elasticity, NPV, beta, ROI) to force an option to fit.
- If the context is incomplete, choose the MOST reasonable answer that fits BOTH the QUESTION and the reliable parts of the CONTEXT,
  and do not contradict the given evidence.

7) STYLE AND FINAL ANSWER FORMAT
- Write your reasoning in plain text sentences (no bullet points, no headings, no Markdown formatting like ** or #).
- Be concise but logically clear.
- At the very end, on a NEW LINE, output the final answer in the exact format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).
- Do NOT wrap this line in Markdown or quotes.
- Do NOT add any extra text before or after the 'Final Answer: X' line.
- The 'Final Answer: X' line MUST appear only once at the very end.

Now, following all of the above rules, reason step by step using the QUESTION and CONTEXT, then give the final answer.
""".strip()

solver_prompt_business = ChatPromptTemplate.from_template(SOLVER_PROMPT_BUSINESS_TMPL)

In [37]:
SOLVER_PROMPT_HISTORY_TMPL = """
You are an expert history exam solver. You specialize in advanced multiple-choice questions
(undergraduate and graduate level) in world history, U.S. history, European history, Asian history,
Latin American and African history, intellectual and cultural history, political history,
economic history, and questions based on primary-source excerpts.

You are given two inputs:

[CONTEXT]
{context}

[QUESTION]
{question}

The CONTEXT acts like textbook + Wikipedia support. The QUESTION is the primary source.
Follow these rules very carefully:

1) PRIORITIZE THE QUESTION TEXT
- First, read the QUESTION (including any passage, quotation, data, and all answer options) very carefully.
- Treat the facts, wording, and time/place cues in the QUESTION as the strongest evidence.
- If the CONTEXT conflicts with the QUESTION, you MUST follow the QUESTION.

2) EXTRACT KEY HISTORICAL CLUES FROM THE QUESTION
When solving the problem, follow this procedure:
1. Look only at the [QUESTION] text first and silently extract 3–5 essential historical clues, such as:
   - time period or approximate dates (e.g., “late 18th century,” “interwar period,” “Post–World War II”),
   - geographic region or polity (e.g., Tang China, Mughal India, Weimar Germany, Cold War United States),
   - type of source (e.g., political speech, treaty, law code, memoir, propaganda, religious text),
   - key themes (e.g., imperialism, industrialization, nationalism, revolution, reform, decolonization, globalization),
   - actors and relationships (e.g., state vs. frontier peoples, colonizer vs. colonized, elite vs. peasantry).
2. Then, look at these clues and the CONTEXT and compare them.
   - If the logic between the QUESTION clues and the CONTEXT conflicts, trust the QUESTION clues.
3. If the clues within textbook-style parts of the CONTEXT and Wikipedia-style parts are conflicted,
   trust the textbook-style exposition in the CONTEXT.

3) USE CONTEXT TO RECALL HISTORY, NOT TO OVERRIDE THE QUESTION
- Use the CONTEXT to recall standard chronology, major events, and characteristic policies or ideas
  of states, movements, and historical figures.
- Do NOT overwrite or ignore specific details given in the QUESTION (e.g., who is speaking, to whom, and in what setting).
- Apply widely accepted historical interpretations unless the CONTEXT or QUESTION clearly specifies a particular viewpoint.

4) INTERPRET SOURCES AND CAUSALITY CAREFULLY
- For questions based on a passage or source:
  - Focus on what the author explicitly states and what is strongly implied by the text.
  - Distinguish between what the author approves of and what they criticize.
  - Note the tone, audience, and purpose (e.g., justify a policy, criticize a ruler, mobilize support).
- Pay careful attention to:
  - cause vs. effect (do not confuse consequences with causes),
  - continuity vs. change over time,
  - similarities vs. differences between regions or periods,
  - whether the question is asking for context, consequence, motivation, or historical significance.

5) COMPARE EACH OPTION CAREFULLY
For EACH answer choice:
- Briefly evaluate why it could be correct or incorrect, using explicit references to:
  - the key clues you extracted from the QUESTION, and
  - relevant facts or patterns from the CONTEXT (chronology, policies, institutions, conflicts).
- When TWO OR MORE options seem partially correct:
  * Prefer the option that:
    - best matches the specific time, place, and actors indicated in the QUESTION,
    - fits the main theme or process the QUESTION is testing,
    - does not rely on anachronistic assumptions or events from a different period,
    - avoids overgeneralization or mixing different regions or eras.

6) AVOID HALLUCINATIONS
- Do NOT introduce additional events, dates, or policies that are not supported by the QUESTION or reliable CONTEXT.
- Do NOT move events to different centuries or regions just to make an option fit.
- Do NOT attribute quotes or ideas to the wrong person or regime.
- If the context is incomplete, choose the MOST reasonable answer that fits BOTH the QUESTION and the reliable parts of the CONTEXT,
  and do not contradict the given evidence.

7) STYLE AND FINAL ANSWER FORMAT
- Write your reasoning in plain text sentences (no bullet points, no headings, no Markdown formatting like ** or #).
- Be concise but logically clear.
- At the very end, on a NEW LINE, output the final answer in the exact format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).
- Do NOT wrap this line in Markdown or quotes.
- Do NOT add any extra text before or after the 'Final Answer: X' line.
- The 'Final Answer: X' line MUST appear only once at the very end.

Now, following all of the above rules, reason step by step using the QUESTION and CONTEXT, then give the final answer.
""".strip()

solver_prompt_history = ChatPromptTemplate.from_template(SOLVER_PROMPT_HISTORY_TMPL)

In [38]:
def solve_mmlu_with_rag_and_wiki(question_text: str):
    # 1) 1단계 LLM: 카테고리 + 키워드
    category, keywords = classify_and_extract_keywords(question_text)
    if category not in vectorstore:
        print(f"[WARN] Unknown category '{category}', fallback to 'business'")
        category = "business"

    # 2) 컨텍스트 구성 (FAISS + Wikipedia)
    context = build_context_for_solver(question_text, category, keywords)

    # 3) 2단계 LLM: CoT로 정답 도출
    if category == "law":
      messages = solver_prompt_law.format_messages(
          context=context,
          question=question_text
      )
    elif category == "philosophy":
      messages = solver_prompt_philosophy.format_messages(
          context=context,
          question=question_text,
      )
    elif category == "psychology":
      messages = solver_prompt_psychology.format_messages(
          context=context,
          question=question_text,
      )
    elif category == "business":
      messages = solver_prompt_business.format_messages(
          context=context,
          question=question_text,
      )
    elif category == "history":
        messages = solver_prompt_history.format_messages(
          context=context,
          question=question_text,
      )

    resp = llm_solver.invoke(messages)
    raw = resp.content.strip()

    # (1) 아래쪽 줄부터 "FINAL ANSWER"가 들어간 줄 찾기 (마크다운 포함 허용)
    for line in raw.splitlines()[::-1]:  # 아래에서부터 검색
        s = line.strip()

        # '**Final Answer: A**' 같이 생긴 것도 잡기 위해 'in' 사용
        if "FINAL ANSWER" in s.upper():
            # 마크다운 볼드 제거
            s = s.replace("**", "").strip()
            # 정규식으로 'Final Answer: X'에서 X만 뽑기
            m = re.search(r"FINAL ANSWER\s*:\s*([A-Z])", s, re.IGNORECASE)
            if m:
                final_letter = m.group(1).upper()
                break

    # 그래도 못 찾으면, 전체 텍스트에서 대문자 한 글자 검색
    if final_letter is None:
        for ch in raw:
            if "A" <= ch <= "Z":
                final_letter = ch
                break

    return {
        "category": category,
        "keywords": keywords,
        "raw_reasoning": raw,       # 디버깅/분석용
        "final_answer": final_letter,
    }

**testset 실행 예시**:
- baseline.csv

In [39]:
cd result

[Errno 2] No such file or directory: 'result'
/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/result


In [40]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/dataset/testset.csv")[26:]  # 질문, 선택지 다 합쳐진 컬럼 가정
QUESTION_COL = "prompts"

results = []
for i, row in df.iterrows():
    q = row[QUESTION_COL]
    print(f"[{i}] Solving: {q[:60]}...")
    out = solve_mmlu_with_rag_and_wiki(q)
    results.append(out["final_answer"])
    df.loc[i, "pred_category"] = out["category"]
    df.loc[i, "kw1"] = out["keywords"][0] if len(out["keywords"]) > 0 else ""
    df.loc[i, "kw2"] = out["keywords"][1] if len(out["keywords"]) > 1 else ""
    df.loc[i, "kw3"] = out["keywords"][2] if len(out["keywords"]) > 2 else ""
    df.loc[i, "rag_cot_answer"] = out["final_answer"]
    df.loc[i, "cot_full"] = out["raw_reasoning"]

df.to_csv("testset_100_answer2.csv", index=False)

[26] Solving: QUESTION27) A man is at home in his apartment, alone, late a...
[27] Solving: QUESTION28) What do Homo sapiens and Australopithecus afaren...
[28] Solving: QUESTION29)This question refers to the following information...
[WARN] page for 'Tang relations' does not exist
[WARN] page for 'frontier peoples' does not exist
[29] Solving: QUESTION30)Homo erectus differed from Homo habilis in which ...
[30] Solving: QUESTION31)During the manic phase of a bipolar disorder, ind...
[WARN] page for 'manic phase' does not exist
[WARN] page for 'high self-esteem' does not exist
[31] Solving: QUESTION32) This question refers to the following informatio...
[32] Solving: QUESTION33) You receive a phone call from Hermann H., age 28...
[WARN] page for 'ethical psychologist' does not exist
[WARN] page for 'referrals' does not exist
[33] Solving: QUESTION34) During the second stage of Kohlberg’s preconvent...
[WARN] page for 'Kohlberg’s theory' does not exist
[WARN] page for 'preconventional le

**정확도**

In [41]:
import pandas as pd

# 1. 파일 경로
GT_PATH = "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/dataset/testset.csv"# 정답 파일
PRED_PATH = "testset_100_answer2.csv"       # 출력 파일

# 2. 컬럼 이름
GT_COL = "answers"               # testset.csv에서 정답 컬럼
PRED_COL = "rag_cot_answer"      # baseline.csv에서 예측 컬럼

# 3. csv 로드
gt_df = pd.read_csv(GT_PATH)[26:]   # 26번 문제부터라고 가정
pred_df = pd.read_csv(PRED_PATH)

# 4. Series 추출
gt = gt_df[GT_COL].astype(str)
pred = pred_df[PRED_COL].astype(str)

# 5. 정규화 함수
def normalize_choice(x: str) -> str:
    x = x.strip().upper()
    for ch in x:
        if "A" <= ch <= "Z":
            return ch
    return x

# 인덱스 리셋이 포인트
gt_norm = gt.apply(normalize_choice).reset_index(drop=True)
pred_norm = pred.apply(normalize_choice).reset_index(drop=True)

print(len(gt_norm), len(pred_norm))  # 둘 다 25 나오는지 체크 한번 해보고

# 6. 정확도 계산
correct = (gt_norm == pred_norm)
accuracy = correct.mean()

print(f"총 문제 수: {len(gt_norm)}")
print(f"정답 개수: {correct.sum()}")
print(f"정확도: {accuracy * 100:.2f}%")

24 24
총 문제 수: 24
정답 개수: 18
정확도: 75.00%
